In [ ]:
# Importation des bibliothèques nécessaires
import cv2
import numpy as np
import time
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

# Chemin pour charger les datasets
results_dir = '/content/drive/MyDrive/formation Datascientest/RVL-CDIP/'

# Charger les datasets enregistrés
dataset = load_from_disk(os.path.join(results_dir, 'val_dataset_ID'))

# Fonction pour redimensionner une image
def resize_image(image):
    return cv2.resize(image, (1500, 1500), interpolation=cv2.INTER_CUBIC)

# Fonction pour traiter une seule image
def process_image(example):
    image = example['image']
    resized_image = resize_image(image)
    return resized_image

# Initialiser le ThreadPoolExecutor pour le traitement parallèle
num_workers = 8  # Ajustez ce nombre en fonction de votre matériel
executor = ThreadPoolExecutor(max_workers=num_workers)

# Liste pour stocker les futures
futures = []

# Démarrer le chronomètre
start_time = time.time()

# Soumettre les tâches au ThreadPoolExecutor
for example in dataset:
    future = executor.submit(process_image, example)
    futures.append(future)

# Collecter les résultats
resized_images = []
for future in tqdm(as_completed(futures), total=len(futures), desc="Redimensionnement des images", unit="image"):
    resized_images.append(future.result())

# Arrêter le chronomètre
end_time = time.time()
processing_time = end_time - start_time

print(f"Temps de traitement total : {round(processing_time / 60, 2)} minutes")

# Exemple de sauvegarde des images redimensionnées (facultatif)
# for i, image in enumerate(resized_images):
#     cv2.imwrite(f'resized_image_{i}.jpg', image)

# Afficher quelques images redimensionnées pour vérification (facultatif)
import matplotlib.pyplot as plt

for i in range(5):
    plt.imshow(cv2.cvtColor(resized_images[i], cv2.COLOR_BGR2RGB))
    plt.title(f'Resized Image {i}')
    plt.show()